In [2]:
%%writefile Multinomial_Framework2.py
import torch
import torch.nn as nn
from torch.utils.data import Dataset 
from torch.utils.data import DataLoader, TensorDataset
from sklearn.ensemble import RandomForestClassifier as SklearnRFC
from sklearn.metrics import accuracy_score
from typing import Union, Tuple


class dataset(Dataset):
    def __init__(self, X, Y):
        # self.X = torch.tensor(X, dtype = torch.float32).to(device)
        # self.Y = torch.tensor(Y, dtype = torch.long).to(device)
        self.X = torch.tensor(X.values if hasattr(X, 'values') else X, dtype=torch.float32)
        self.Y = torch.tensor(Y.values if hasattr(Y, 'values') else Y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.Y[index]

class RandomForestWrapper():
    '''Wrapper for sklearn's RandomForestClassifier with consistent API.'''

    def __init__(self, n_estimators=200, max_depth=None, min_samples_split=2,
                 class_weight='balanced', random_state=42, n_jobs=-1):

        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.class_weight = class_weight
        self.random_state = random_state
        self.n_jobs = n_jobs
        
        self.model = SklearnRFC(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            class_weight=class_weight,
            random_state=random_state,
            n_jobs=n_jobs
        )

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def evaluate(self, X, y):
        preds = self.predict(X)
        acc = accuracy_score(y, preds)
        return {"accuracy": round(acc*100,3)}

class NN_Wrapper():
    '''Wrapper for PyTorch neural networks with the same interface as sklearn models.'''
    
    def __init__(self, model: nn.Module, criterion, optimizer: torch.optim.Optimizer,
                 epochs: int = 10, batch_size: int = 256, device: str = None):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.epochs = epochs
        self.batch_size = batch_size
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

    def fit(self, X, y):
        total_acc_train_plot = []
        X_tensor = torch.tensor(X.values if hasattr(X, 'values') else X, dtype=torch.float32)
        y_tensor = torch.tensor(y.values if hasattr(y, 'values') else y, dtype=torch.long)
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        self.model.train()
        for epoch in range(self.epochs):
            total_loss = 0
            total_acc = 0
            for inputs, labels in dataloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                preds = self.model(inputs)     
                
                loss = self.criterion(preds, labels)
                
                total_loss += loss.item()
                
                predicted_classes = torch.argmax(preds,dim=1)
                
                acc = (predicted_classes == labels).sum().item()

                total_acc += acc
                
                self.optimizer.zero_grad()
                
                loss.backward()
                
                self.optimizer.step()
                
            total_acc_train_plot.append(round(total_acc/X.__len__() * 100, 4))    
            print(f"Epoch [{epoch+1}/{self.epochs}] | Loss: {round(total_loss/len(dataloader) * 100, 4)} | Train Acc: {round(total_acc/X.__len__() * 100, 4)}")
        return self, total_acc_train_plot

    def predict(self, testing_dataloader):
        #X_tensor = torch.tensor(X.values if hasattr(X, 'values') else X, dtype=torch.float32)
        #self.model.eval()
        total_loss_test = 0
        total_acc_test = 0
        total_acc_test_plot = []
        with torch.no_grad():
            for inputs, labels in testing_dataloader:
                preds = self.model(inputs)
                loss = self.criterion(preds,labels)
                total_loss_test += loss
                predicted_classes = torch.argmax(preds, dim=1).cpu().numpy()
                acc = (predicted_classes == labels).sum().item()
                total_acc_test += acc
                total_acc_test_plot.append(round(total_acc_test/len(testing_dataloader.dataset) * 100, 4)) 
        test_size = len(testing_dataloader.dataset)
        avg_acc = total_acc_test / test_size
        avg_loss = total_loss_test / len(testing_dataloader)
        
        print(f'Test Accuracy: {avg_acc*100:.2f}% | Test Loss: {avg_loss*100:.2f}')
        return total_acc_test_plot

class Diabetes_Predictor_NN(nn.Module):
    def __init__(self, input_dim, num_classes=3, hidden_neurons=3):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(input_dim, hidden_neurons),
            nn.ReLU(),

            nn.Linear(hidden_neurons,32),
            nn.ReLU(),

            nn.Linear(32, 64),
            nn.ReLU(),

            nn.Linear(64,32),
            nn.ReLU(),

            nn.Linear(32,16),
            nn.ReLU(),

            nn.Linear(16,num_classes),
        )
    def forward(self,x):
        x = self.linear_layer_stack(x)
        return x

Overwriting Multinomial_Framework2.py
